In [1]:
import sqlite3 
import pandas as pd
import pandas_profiling as pp
import numpy as np

In [2]:
conn = sqlite3.connect('traffic.db')

In [3]:
sql_master = """SELECT * FROM sqlite_master"""
pd.read_sql_query(sql_master, conn)

type     name tbl_name  rootpage  \
0  table  traffic  traffic         2   

                                                 sql  
0  CREATE TABLE traffic(\n  "SeqID" TEXT,\n  "Dat...

In [4]:
sql_all = "SELECT * FROM traffic"
all_df = pd.read_sql_query(sql_all, conn)

In [5]:
pp.ProfileReport(all_df)

/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/image.py:863: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=-0.5, right=-0.5
  self.axes.set_xlim((xmin, xmax), auto=None)
/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/image.py:865: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=-0.5, top=-0.5
  self.axes.set_ylim((ymin, ymax), auto=None)


ZeroDivisionError: float division by zero

In [13]:
all_df.describe()

SeqID Date Of Stop Time Of Stop  \
count                                1529036      1529036      1529036   
unique                                867384         2710         1440   
top     33c49de3-9e36-4f36-9326-b59a95e86fe8   03/17/2015     23:30:00   
freq                                      59         1281         2312   

         Agency              SubAgency  \
count   1529036                1529036   
unique        1                      9   
top         MCP  4th District, Wheaton   
freq    1529036                 375716   

                                              Description  \
count                                             1529036   
unique                                              13987   
top     DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC...   
freq                                               125683   

                   Location Latitude Longitude Accident      ...        \
count               1529036  1529036   1529036  1529036      ...         
unique               210296   298798    339125        6      ...         
top     WAYNE AVE @ DALE DR        0         0       No      ...         
freq                   2172   110485    110485  1482066      ...         

          Driver City Driver State DL State        Arrest Type Geolocation  \
count         1529036      1529036  1529036            1529036     1529036   
unique           7940           69       71                 19      749951   
top     SILVER SPRING           MD       MD  A - Marked Patrol  (0.0, 0.0)   
freq           380216      1379705  1329309            1267088      110485   

       Council Districts Councils Communities Zip Codes Municipalities  
count            1529036  1529036     1529036   1529036        1529036  
unique                 6        6          63       145             25  
top                    4        4          36        17              1  
freq              334292   334292      184196    131235        1161876  

[4 rows x 41 columns]

In [ ]:
all_df['Violation Type'].value_counts()

In [61]:
grouped_race_violation = all_df.groupby(['Race', 'Violation Type']).size().astype(float)

In [20]:
all_df['Arrest Type'].value_counts()

A - Marked Patrol                         1267088
Q - Marked Laser                           139542
B - Unmarked Patrol                         48181
S - License Plate Recognition               14509
O - Foot Patrol                             14076
L - Motorcycle                              13031
E - Marked Stationary Radar                  9511
G - Marked Moving Radar (Stationary)         8210
R - Unmarked Laser                           6140
I - Marked Moving Radar (Moving)             2476
M - Marked (Off-Duty)                        2272
H - Unmarked Moving Radar (Stationary)       1115
F - Unmarked Stationary Radar                 837
J - Unmarked Moving Radar (Moving)            654
C - Marked VASCAR                             483
P - Mounted Patrol                            338
D - Unmarked VASCAR                           302
N - Unmarked (Off-Duty)                       216
K - Aircraft Assist                            55
Name: Arrest Type, dtype: int64

In [62]:
sum(grouped_race_violation['WHITE'])

541783.0

In [63]:
grouped_normalize = grouped_race_violation.copy()

for race in grouped_race_violation.index.levels[0]:
    race_viol = sum(grouped_race_violation[race])
    for viol in grouped_race_violation.index.levels[1]:
        try:
            grouped_normalize[race][viol] = grouped_race_violation[race][viol] / float(race_viol)
        except:
            pass

In [64]:
grouped_normalize

Race             Violation Type
ASIAN            Citation          0.377285
                 ESERO             0.051709
                 SERO              0.000379
                 Warning           0.570627
BLACK            Citation          0.466892
                 ESERO             0.047441
                 SERO              0.000529
                 Warning           0.485139
HISPANIC         Citation          0.518475
                 ESERO             0.062510
                 SERO              0.000826
                 Warning           0.418189
NATIVE AMERICAN  Citation          0.410354
                 ESERO             0.068182
                 Warning           0.521465
OTHER            Citation          0.388484
                 ESERO             0.045180
                 SERO              0.000516
                 Warning           0.565821
WHITE            Citation          0.425370
                 ESERO             0.041002
                 SERO              0.000546


In [71]:
all_df.groupby(['Race', 'Arrest Type']).size().astype(float)

Race   Arrest Type                           
ASIAN  A - Marked Patrol                          73514.0
       B - Unmarked Patrol                         2400.0
       C - Marked VASCAR                             41.0
       D - Unmarked VASCAR                           22.0
       E - Marked Stationary Radar                  731.0
       F - Unmarked Stationary Radar                 50.0
       G - Marked Moving Radar (Stationary)         543.0
       H - Unmarked Moving Radar (Stationary)        60.0
       I - Marked Moving Radar (Moving)             114.0
       J - Unmarked Moving Radar (Moving)            37.0
       K - Aircraft Assist                            1.0
       L - Motorcycle                               838.0
       M - Marked (Off-Duty)                        169.0
       N - Unmarked (Off-Duty)                        7.0
       O - Foot Patrol                              800.0
       P - Mounted Patrol                            24.0
       Q - Marked Laser   

In [78]:
all_df['Arrest Type'].value_counts(True)

A - Marked Patrol                         0.828684
Q - Marked Laser                          0.091261
B - Unmarked Patrol                       0.031511
S - License Plate Recognition             0.009489
O - Foot Patrol                           0.009206
L - Motorcycle                            0.008522
E - Marked Stationary Radar               0.006220
G - Marked Moving Radar (Stationary)      0.005369
R - Unmarked Laser                        0.004016
I - Marked Moving Radar (Moving)          0.001619
M - Marked (Off-Duty)                     0.001486
H - Unmarked Moving Radar (Stationary)    0.000729
F - Unmarked Stationary Radar             0.000547
J - Unmarked Moving Radar (Moving)        0.000428
C - Marked VASCAR                         0.000316
P - Mounted Patrol                        0.000221
D - Unmarked VASCAR                       0.000198
N - Unmarked (Off-Duty)                   0.000141
K - Aircraft Assist                       0.000036
Name: Arrest Type, dtype: float

In [76]:
len(all_df)

1529036

In [83]:
descrips = all_df.Description.value_counts()

In [93]:
descrips.values

array([125683,  71761,  48988, ...,      1,      1,      1])

In [111]:
np.percentile(descrips.values, 90, axis=0)

21.0

In [106]:
import matplotlib.pyplot as plt
%matplotlib inline

In [116]:
a = sorted(descrips.values)
cumsum_descrips = [sum(a[:i]) for i in range(len(a))]
cumsum_descrips

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [122]:
x_percentile = (np.array(range(len(a)))+1)/len(a)


plt.plot(x_percentile, cumsum_descrips)

1.0

In [145]:
all_df.Description.value_counts().index[1]

'FAILURE TO DISPLAY REGISTRATION CARD UPON DEMAND BY POLICE OFFICER'

In [148]:
viol_type = 'FAILURE TO DISPLAY REGISTRATION CARD UPON DEMAND BY POLICE OFFICER'

all_df[all_df['Description'].isin([viol_type])].groupby('Race').size()

Race
ASIAN               5100
BLACK              24133
HISPANIC           13885
NATIVE AMERICAN      141
OTHER               4031
WHITE              24471
dtype: int64